# Arabic Tweets Sentiment Analysis with Stacking a Meta Learner on 2 classifiers - (countvectorizer + embeddings based classifiers) 
Stacking a meta learner (a 3rd model which is LR) on 2 different models: 
1. Pre-trained AraBERT model for vectorization/embeddings and Catboost for sentiment classification inside Vantage
2. LR based sentiment classifier only on the Count Vectorizer based embeddings inside vantage

## 1. Import Libraries

In [3]:
import pandas as pd
import numpy as np
import torch
import re
import os
import nltk
import getpass as gp
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import onnxruntime as ort
from tqdm import tqdm
import teradataml as tdml
import catboost
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Download stopwords if not already downloaded
nltk.download('stopwords', quiet=True)
arabic_stopwords = stopwords.words('arabic')

In [ ]:
td_context = tdml.create_context(host="host address added here",
                            username="username added here",
                            password = gp.getpass(prompt='Password:')
                           )

## 2. Load and Preprocess Dataset

In [ ]:
# Load dataset
df = pd.read_csv('../Datasets/Arabic Sentiment Analysis Dataset - SS2030.csv')
df = df.reset_index().rename(columns={'index': 'tid', 'text': 'docstring'})
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (4252, 3)


,tid,docstring,Sentiment
0,0,حقوق المرأة 💚💚💚 https://t.co/Mzf90Ta5g1,1
1,1,RT @___IHAVENOIDEA: حقوق المرأة في الإسلام. ht...,1
2,2,RT @saud_talep: Retweeted لجنة التنمية بشبرا (...,1
3,3,RT @MojKsa: حقوق المرأة التي تضمنها لها وزارة ...,1
4,4,RT @abm112211: ولي امر الزوجة او ولي الزوجة او...,1


In [5]:
# Preprocessing function
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', text)
    # Normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove stopwords
    words = text.split()
    words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(words)

# Clean the text
df['cleaned_doc'] = df['docstring'].apply(clean_text)
df = df[df['cleaned_doc'].notna() & (df['cleaned_doc'].astype(str).str.strip() != '')]
df = df[['tid', 'cleaned_doc', 'Sentiment']]
print(f"\nSentiment distribution:\n{df['Sentiment'].value_counts()}")
df.head()


Sentiment distribution:
Sentiment
1    2436
0    1815
Name: count, dtype: int64


,tid,cleaned_doc,Sentiment
0,0,حقوق المرأة,1
1,1,حقوق المرأة الإسلام,1
2,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1
3,3,حقوق المرأة تضمنها وزارة العدل,1
4,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1


In [6]:
tdml.copy_to_sql(df, table_name="sentiment_analysis_data_cleaned", if_exists="replace", primary_index="tid")
tdml.execute_sql("SELECT TOP 1 * FROM sentiment_analysis_data_cleaned").fetchall()

[[469,
  'الولاية عبودية ضد المرأة استغلها ذكور للتسلط عليها سعوديات نطلب اسقاط الولايه',
  1]]

## 3. Generating Model 1 - AraBERT + Catboost Classifier

In [7]:
# Download model if needed (run only once)
# Uncomment to download:
# model_name = "PRAli22/AraBert-Arabic-Sentiment-Analysis"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer.save_pretrained("./arabert_sentiment_model")
# model.save_pretrained("./arabert_sentiment_model")

In [8]:
# Load model from local directory
model_dir = "./arabert_sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
model.eval()
print("Model loaded successfully")

Model loaded successfully


In [9]:
# Function to get embeddings
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model.bert(**inputs)  # Get BERT base outputs
        # Use [CLS] token embedding (first token)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy().flatten()
    return embeddings

In [10]:
# Generate embeddings for all texts
print("Generating embeddings...")
embeddings_list = []
for text in tqdm(df['cleaned_doc'], desc="Creating embeddings"):
    emb = get_embeddings(text)
    embeddings_list.append(emb)

Generating embeddings...


Creating embeddings:   0%|          | 0/4251 [00:00<?, ?it/s]

Creating embeddings: 100%|██████████| 4251/4251 [03:58<00:00, 17.80it/s]


In [11]:
# Convert list of embeddings to numpy array
embeddings_array = np.array(embeddings_list, dtype=np.float32)

# Create column names for embeddings (emb_0, emb_1, ..., emb_767)
embedding_columns = [f'emb_{i}' for i in range(embeddings_array.shape[1])]

# Create dataframe with embeddings as separate columns
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns, dtype=np.float32)

# Add tid, text, and sentiment columns
embeddings_df.insert(0, 'tid', df['tid'].values)
embeddings_df.insert(1, 'cleaned_doc', df['cleaned_doc'].values)
embeddings_df.insert(2, 'Sentiment', df['Sentiment'].values)

print(f"\nDataframe shape: {embeddings_df.shape}")
print(f"Number of embedding columns: {len(embedding_columns)}")
print(f"\nColumn names: {embeddings_df.columns.tolist()[:5]}... (first 5)")
print(f"\nData types:\n{embeddings_df.dtypes.head(10)}")
print("\nFirst few rows:")
embeddings_df.head()


Dataframe shape: (4251, 771)
Number of embedding columns: 768

Column names: ['tid', 'cleaned_doc', 'Sentiment', 'emb_0', 'emb_1']... (first 5)

Data types:
tid              int64
cleaned_doc     object
Sentiment        int64
emb_0          float32
emb_1          float32
emb_2          float32
emb_3          float32
emb_4          float32
emb_5          float32
emb_6          float32
dtype: object

First few rows:


,tid,cleaned_doc,Sentiment,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,0,حقوق المرأة,1,0.915905,-0.582944,1.188565,-1.250446,1.040004,-1.015085,-3.424124,...,1.092006,-0.102470,-0.293598,-0.512675,-0.347759,-0.649128,0.602792,0.992201,-2.023002,0.048379
1,1,حقوق المرأة الإسلام,1,0.769661,-1.120466,0.702905,-1.390138,0.862794,-0.822431,-3.307889,...,0.569811,-0.492879,-0.406484,-0.717845,0.276217,0.050306,0.271503,1.274817,-1.587476,0.246192
2,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1,1.582610,1.252096,1.246331,-2.210566,-0.014998,0.320480,-2.154431,...,0.167818,0.823797,0.896441,-0.089060,-0.165477,-0.669874,0.585659,0.447458,-0.787317,-0.296942
3,3,حقوق المرأة تضمنها وزارة العدل,1,0.835202,-1.044387,0.848940,-0.381577,0.344434,-1.078913,-3.103493,...,0.612538,-0.593038,0.565501,-0.508824,0.036350,0.635917,-0.261125,0.594527,-1.912073,0.470904
4,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1,0.394898,0.734406,0.526005,-1.466056,-0.266501,1.107806,-1.597542,...,0.018454,-1.743913,0.732452,-2.575912,0.220516,0.800939,0.618779,0.600923,1.654606,-0.109317


### 3a. Lets push the embeddings data to Vantage

In [12]:
tdml.copy_to_sql(embeddings_df, table_name="arabic_tweets_embedding_data", if_exists="replace", primary_index='tid')

In [13]:
tdf = tdml.DataFrame("arabic_tweets_embedding_data")
tdf.head()

cleaned_doc  Sentiment     emb_0     emb_1     emb_2     emb_3     emb_4     emb_5     emb_6     emb_7     emb_8     emb_9    emb_10    emb_11    emb_12    emb_13    emb_14    emb_15    emb_16    emb_17    emb_18    emb_19    emb_20    emb_21    emb_22    emb_23    emb_24    emb_25    emb_26    emb_27    emb_28    emb_29    emb_30    emb_31    emb_32    emb_33    emb_34    emb_35    emb_36    emb_37    emb_38    emb_39    emb_40    emb_41    emb_42    emb_43    emb_44    emb_45    emb_46    emb_47    emb_48    emb_49    emb_50    emb_51    emb_52    emb_53    emb_54    emb_55    emb_56    emb_57    emb_58    emb_59    emb_60    emb_61    emb_62    emb_63    emb_64    emb_65    emb_66    emb_67    emb_68    emb_69    emb_70    emb_71    emb_72    emb_73    emb_74    emb_75    emb_76    emb_77    emb_78    emb_79    emb_80    emb_81    emb_82    emb_83    emb_84    emb_85    emb_86    emb_87    emb_88    emb_89    emb_90    emb_91    emb_92    emb_93    emb_94    emb_95    emb_96    emb_97    emb_98    emb_99   emb_100   emb_101   emb_102   emb_103   emb_104   emb_105   emb_106   emb_107   emb_108   emb_109   emb_110   emb_111   emb_112   emb_113   emb_114   emb_115   emb_116   emb_117   emb_118   emb_119   emb_120   emb_121   emb_122   emb_123   emb_124   emb_125   emb_126   emb_127   emb_128   emb_129   emb_130   emb_131   emb_132   emb_133   emb_134   emb_135   emb_136   emb_137   emb_138   emb_139   emb_140   emb_141   emb_142   emb_143   emb_144   emb_145   emb_146   emb_147   emb_148   emb_149   emb_150   emb_151   emb_152   emb_153   emb_154   emb_155   emb_156   emb_157   emb_158   emb_159   emb_160   emb_161   emb_162   emb_163   emb_164   emb_165   emb_166   emb_167   emb_168   emb_169   emb_170   emb_171   emb_172   emb_173   emb_174   emb_175   emb_176   emb_177   emb_178   emb_179   emb_180   emb_181   emb_182   emb_183   emb_184   emb_185   emb_186   emb_187   emb_188   emb_189   emb_190   emb_191   emb_192   emb_193   emb_194   emb_195   emb_196   emb_197   emb_198   emb_199   emb_200   emb_201   emb_202   emb_203   emb_204   emb_205   emb_206   emb_207   emb_208   emb_209   emb_210   emb_211   emb_212   emb_213   emb_214   emb_215   emb_216   emb_217   emb_218   emb_219   emb_220   emb_221   emb_222   emb_223   emb_224   emb_225   emb_226   emb_227   emb_228   emb_229   emb_230   emb_231   emb_232   emb_233   emb_234   emb_235   emb_236   emb_237   emb_238   emb_239   emb_240   emb_241   emb_242   emb_243   emb_244   emb_245   emb_246   emb_247   emb_248   emb_249   emb_250   emb_251   emb_252   emb_253   emb_254   emb_255   emb_256   emb_257   emb_258   emb_259   emb_260   emb_261   emb_262   emb_263   emb_264   emb_265   emb_266   emb_267   emb_268   emb_269   emb_270   emb_271   emb_272   emb_273   emb_274   emb_275   emb_276   emb_277   emb_278   emb_279   emb_280   emb_281   emb_282   emb_283   emb_284   emb_285   emb_286   emb_287   emb_288   emb_289   emb_290   emb_291   emb_292   emb_293   emb_294   emb_295   emb_296   emb_297   emb_298   emb_299   emb_300   emb_301   emb_302   emb_303   emb_304   emb_305   emb_306   emb_307   emb_308   emb_309   emb_310   emb_311   emb_312   emb_313   emb_314   emb_315   emb_316   emb_317   emb_318   emb_319   emb_320   emb_321   emb_322   emb_323   emb_324   emb_325   emb_326   emb_327   emb_328   emb_329   emb_330   emb_331   emb_332   emb_333   emb_334   emb_335   emb_336   emb_337   emb_338   emb_339   emb_340   emb_341   emb_342   emb_343   emb_344   emb_345   emb_346   emb_347   emb_348   emb_349   emb_350   emb_351   emb_352   emb_353   emb_354   emb_355   emb_356   emb_357   emb_358   emb_359   emb_360   emb_361   emb_362   emb_363   emb_364   emb_365   emb_366   emb_367   emb_368   emb_369   emb_370   emb_371   emb_372   emb_373   emb_374   emb_375   emb_376   emb_377   emb_378   emb_379   emb_380   emb_381   emb_382   emb_383   emb_384   emb_385   emb_386   emb_387   emb_388   emb_389   emb_390   emb_391   emb_392   emb_393   emb_394   emb_395   emb_396   emb_3

Splitting the training and test dataset

In [14]:
TrainTestSplit_out = tdml.TrainTestSplit(
                                    data = tdf,
                                    id_column = "tid",
                                    train_size = 0.70,
                                    test_size = 0.30,
                                    seed = 21
)

# Split into 2 virtual dataframes
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

In [15]:
df_train.shape

(2975, 771)

Pulling out training set from Vantage to local disk

In [16]:
df_train.to_csv("trainset_model1.csv", all_rows=True, fastexport=False)
training_dataset = pd.read_csv("trainset_model1.csv")
training_dataset.head()


Data is successfully exported into trainset_model1.csv


,tid,cleaned_doc,Sentiment,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,1,حقوق المرأة الإسلام,1,0.769661,-1.120466,0.702905,-1.390138,0.862794,-0.822431,-3.307889,...,0.569811,-0.492879,-0.406484,-0.717845,0.276217,0.050306,0.271503,1.274817,-1.587476,0.246192
1,9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق ا...,1,2.687572,-0.492569,0.918326,-1.587362,0.456493,0.878994,-1.847191,...,-0.507663,-0.319201,0.755966,-0.639121,-0.377064,1.697193,-0.357151,0.048731,0.910484,-1.209859
2,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1,0.394898,0.734406,0.526005,-1.466056,-0.266501,1.107806,-1.597542,...,0.018454,-1.743913,0.732452,-2.575912,0.220516,0.800939,0.618779,0.600923,1.654606,-0.109317
3,0,حقوق المرأة,1,0.915905,-0.582944,1.188565,-1.250446,1.040004,-1.015085,-3.424124,...,1.092006,-0.102470,-0.293598,-0.512675,-0.347759,-0.649128,0.602792,0.992201,-2.023002,0.048379
4,3,حقوق المرأة تضمنها وزارة العدل,1,0.835202,-1.044387,0.848940,-0.381577,0.344434,-1.078913,-3.103493,...,0.612538,-0.593038,0.565501,-0.508824,0.036350,0.635917,-0.261125,0.594527,-1.912073,0.470904


Training the model through 5-Fold Cross Validation for Meta Learner

In [17]:
from sklearn.model_selection import StratifiedKFold

print(f"Training set shape: {training_dataset.shape}")
print(f"Columns: {training_dataset.columns.tolist()[:10]}...")  # Show first 10 columns

# ============================================================================
# STEP 3: Prepare features and labels
# ============================================================================
# Extract row IDs, text, and labels
train_ids = training_dataset['tid'].values
train_text = training_dataset['cleaned_doc'].values
y_train = training_dataset['Sentiment'].values

# Extract embedding features (assuming columns emb_0, emb_1, ..., emb_N)
# Adjust the column names based on your actual embedding column names
embedding_cols = [col for col in training_dataset.columns if col.startswith('emb_')]
X_train = training_dataset[embedding_cols].values

print(f"\nTraining set size: {len(X_train)}")
print(f"Number of embedding features: {X_train.shape[1]}")
print(f"Class distribution: {np.bincount(y_train)}")

# ============================================================================
# STEP 4: Define your model
# ============================================================================
# You can use CatBoost, XGBoost, or any other classifier

model = catboost.CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    early_stopping_rounds=50,
    verbose=False,
    random_state=42
)

print("\nModel defined")

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize arrays to store out-of-fold predictions
oof_probs = np.zeros((len(X_train), 2))  # Probabilities for both classes [prob_0, prob_1]
oof_preds = np.zeros(len(X_train))       # Predicted labels

print(f"\nStarting {n_folds}-fold cross-validation...")
print("="*70)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"\n{'='*70}")
    print(f"FOLD {fold}/{n_folds}")
    print(f"{'='*70}")
    print(f"Train size: {len(train_idx)}, Validation size: {len(val_idx)}")
    
    # Split data for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    
    # Train the model on this fold
    print(f"Training model...")
    model.fit(X_fold_train, y_fold_train)
    
    # Get predictions on validation fold
    oof_probs[val_idx] = model.predict_proba(X_fold_val)
    oof_preds[val_idx] = model.predict(X_fold_val)
    
    # Calculate fold metrics
    fold_acc = accuracy_score(y_fold_val, oof_preds[val_idx])
    fold_f1 = f1_score(y_fold_val, oof_preds[val_idx])
    fold_auc = roc_auc_score(y_fold_val, oof_probs[val_idx, 1])
    
    print(f"Fold {fold} - Accuracy: {fold_acc:.4f}, F1: {fold_f1:.4f}, AUC: {fold_auc:.4f}")

print("\n" + "="*70)
print("Cross-validation complete!")
print("="*70)


Training set shape: (2975, 771)
Columns: ['tid', 'cleaned_doc', 'Sentiment', 'emb_0', 'emb_1', 'emb_2', 'emb_3', 'emb_4', 'emb_5', 'emb_6']...

Training set size: 2975
Number of embedding features: 768
Class distribution: [1251 1724]

Model defined

Starting 5-fold cross-validation...

FOLD 1/5
Train size: 2380, Validation size: 595
Training model...
Fold 1 - Accuracy: 0.9042, F1: 0.9175, AUC: 0.9686

FOLD 2/5
Train size: 2380, Validation size: 595
Training model...
Fold 2 - Accuracy: 0.8924, F1: 0.9086, AUC: 0.9633

FOLD 3/5
Train size: 2380, Validation size: 595
Training model...
Fold 3 - Accuracy: 0.9025, F1: 0.9171, AUC: 0.9632

FOLD 4/5
Train size: 2380, Validation size: 595
Training model...
Fold 4 - Accuracy: 0.9092, F1: 0.9231, AUC: 0.9672

FOLD 5/5
Train size: 2380, Validation size: 595
Training model...
Fold 5 - Accuracy: 0.9109, F1: 0.9240, AUC: 0.9745

Cross-validation complete!


In [18]:
oof_acc = accuracy_score(y_train, oof_preds)
oof_f1 = f1_score(y_train, oof_preds)
oof_auc = roc_auc_score(y_train, oof_probs[:, 1])

print("\n" + "="*70)
print("OVERALL OUT-OF-FOLD PERFORMANCE")
print("="*70)
print(f"Accuracy:  {oof_acc:.4f}")
print(f"F1-Score:  {oof_f1:.4f}")
print(f"AUC:       {oof_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_train, oof_preds))

predicted_label_prob = np.where(
    oof_preds == 1, 
    oof_probs[:, 1],  # If predicted 1, take prob of class 1
    oof_probs[:, 0]   # If predicted 0, take prob of class 0
)

# Create the final DataFrame
m1_result_df = pd.DataFrame({
    'tid': train_ids,
    'cleaned_doc': train_text,
    'actual_label': y_train,
    'm1_predicted_label': oof_preds.astype(int),
    'm1_predicted_label_probability': predicted_label_prob,
    'm1_prob_class_0': oof_probs[:, 0],
    'm1_prob_class_1': oof_probs[:, 1]
})

print("\n" + "="*70)
print("OUT-OF-FOLD PREDICTIONS DATAFRAME")
print("="*70)
print(f"Shape: {m1_result_df.shape}")
print(f"\nColumns: {list(m1_result_df.columns)}")
print("\nFirst 10 rows:")
m1_result_df.head(10)


OVERALL OUT-OF-FOLD PERFORMANCE
Accuracy:  0.9039
F1-Score:  0.9181
AUC:       0.9677

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      1251
           1       0.91      0.93      0.92      1724

    accuracy                           0.90      2975
   macro avg       0.90      0.90      0.90      2975
weighted avg       0.90      0.90      0.90      2975


OUT-OF-FOLD PREDICTIONS DATAFRAME
Shape: (2975, 7)

Columns: ['tid', 'cleaned_doc', 'actual_label', 'm1_predicted_label', 'm1_predicted_label_probability', 'm1_prob_class_0', 'm1_prob_class_1']

First 10 rows:


,tid,cleaned_doc,actual_label,m1_predicted_label,m1_predicted_label_probability,m1_prob_class_0,m1_prob_class_1
0,1,حقوق المرأة الإسلام,1,1,0.996209,0.003791,0.996209
1,9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق ا...,1,1,0.987403,0.012597,0.987403
2,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1,1,0.980837,0.019163,0.980837
3,0,حقوق المرأة,1,1,0.994422,0.005578,0.994422
4,3,حقوق المرأة تضمنها وزارة العدل,1,1,0.992388,0.007612,0.992388
5,11,محاولات بائسه لاقصاء دور الام السعودية ودورها ...,1,1,0.967114,0.032886,0.967114
6,12,كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، فالل...,1,1,0.511258,0.488742,0.511258
7,14,تعرف ان المرأة عندها حقوق؟ يوم يطبعون ورقة,1,1,0.990851,0.009149,0.990851
8,5,انا كامرأة يوجعني حزن نساء العالم، سواء تعنفت،...,1,1,0.995383,0.004617,0.995383
9,13,الحقوق او المزعوم انها حقوق عبارة صفر مكعب مال...,1,1,0.924092,0.075908,0.924092


### 3b. Evaluating Catboost on the Test Set

In [19]:
test_dataset = df_test.to_csv("testset_model1.csv", all_rows=True, fastexport=False)
test_dataset = pd.read_csv("testset_model1.csv")
test_dataset.head()


Data is successfully exported into testset_model1.csv


,tid,cleaned_doc,Sentiment,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,...,emb_758,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767
0,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1,1.582610,1.252096,1.246331,-2.210566,-0.014998,0.320480,-2.154431,...,0.167818,0.823797,0.896441,-0.089060,-0.165477,-0.669874,0.585659,0.447458,-0.787317,-0.296942
1,10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وه...,1,0.119550,0.641493,1.329657,-1.626576,-0.244940,0.976920,-2.952441,...,-0.843681,0.202292,0.983413,-0.280799,-1.034488,2.325436,0.122212,0.485375,0.694378,-1.464473
2,7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المت...,1,-0.639943,-1.222346,1.922086,-1.380060,0.203868,-0.311772,-2.933300,...,0.494455,0.261476,0.724736,-1.346588,-0.735644,2.314811,0.762845,0.567371,1.422184,-1.089902
3,6,اومن حقوق المرأة أنك تفتحلها الباب,1,-0.815878,-0.573813,0.875430,0.729033,-1.369993,-0.599379,-2.230564,...,0.301263,0.136895,0.830590,-0.223158,0.011948,1.432901,-0.107069,1.015441,1.372740,0.151745
4,8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,-0.694978,-0.020998,1.735893,-0.455287,-0.197177,0.546692,-2.294147,...,0.633531,-0.452604,0.351147,-0.382032,-0.814823,1.567826,0.536734,1.648678,-1.004110,-0.132113


In [33]:
print(f"Test set shape: {test_dataset.shape}")

# Extract test IDs, text, and labels
test_ids = test_dataset['tid'].values
test_text = test_dataset['cleaned_doc'].values
y_test = test_dataset['Sentiment'].values

# Extract embedding features (same columns as training)
X_test = test_dataset[embedding_cols].values

print(f"Test set size: {len(X_test)}")
print(f"Number of features: {X_test.shape[1]}")
print(f"Class distribution: {np.bincount(y_test)}")

# ============================================================================
# STEP 3: Make predictions on test set
# ============================================================================
print("\n" + "="*70)
print("MAKING PREDICTIONS ON TEST SET")
print("="*70)

# Get predictions and probabilities
test_probs = model.predict_proba(X_test)
test_preds = model.predict(X_test)

print(f"✓ Generated predictions for {len(test_preds)} samples")

# ============================================================================
# STEP 4: Calculate test set performance
# ============================================================================

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

test_acc = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds)
test_auc = roc_auc_score(y_test, test_probs[:, 1])

print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)
print(f"Accuracy:  {test_acc:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
print(f"AUC:       {test_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_preds))

print("\nConfusion Matrix:")
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, test_preds)
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives:  {cm[1,1]}")

# ============================================================================
# STEP 5: Create DataFrame with test predictions and probabilities
# ============================================================================
# For each row, store the probability of the PREDICTED label
predicted_label_prob = np.where(
    test_preds == 1, 
    test_probs[:, 1],  # If predicted 1, take prob of class 1
    test_probs[:, 0]   # If predicted 0, take prob of class 0
)

# Create the test results DataFrame
m1_test_result_df = pd.DataFrame({
    'tid': test_ids,
    'cleaned_doc': test_text,
    'actual_label': y_test,
    'm1_predicted_label': test_preds.astype(int),
    'm1_predicted_label_probability': predicted_label_prob,
    # Include both probabilities for meta-learner
    'm1_prob_class_0': test_probs[:, 0],
    'm1_prob_class_1': test_probs[:, 1]
})

print("\n" + "="*70)
print("TEST SET PREDICTIONS DATAFRAME")
print("="*70)
print(f"Shape: {m1_test_result_df.shape}")
print(f"\nColumns: {list(m1_test_result_df.columns)}")
print("\nFirst 10 rows:")
m1_test_result_df.head(10)


Test set shape: (1276, 771)
Test set size: 1276
Number of features: 768
Class distribution: [564 712]

MAKING PREDICTIONS ON TEST SET
✓ Generated predictions for 1276 samples

TEST SET PERFORMANCE
Accuracy:  0.8934
F1-Score:  0.9042
AUC:       0.9634

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88       564
           1       0.91      0.90      0.90       712

    accuracy                           0.89      1276
   macro avg       0.89      0.89      0.89      1276
weighted avg       0.89      0.89      0.89      1276


Confusion Matrix:
[[498  66]
 [ 70 642]]

True Negatives:  498
False Positives: 66
False Negatives: 70
True Positives:  642

TEST SET PREDICTIONS DATAFRAME
Shape: (1276, 7)

Columns: ['tid', 'cleaned_doc', 'actual_label', 'm1_predicted_label', 'm1_predicted_label_probability', 'm1_prob_class_0', 'm1_prob_class_1']

First 10 rows:


,tid,cleaned_doc,actual_label,m1_predicted_label,m1_predicted_label_probability,m1_prob_class_0,m1_prob_class_1
0,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1,1,0.947813,0.052187,0.947813
1,10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وه...,1,0,0.565880,0.565880,0.434120
2,7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المت...,1,1,0.851383,0.148617,0.851383
3,6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.991968,0.008032,0.991968
4,8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.782267,0.217733,0.782267
5,16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابت...,1,0,0.875465,0.875465,0.124535
6,27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حال...,1,1,0.988273,0.011727,0.988273
7,19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا ال...,1,1,0.932542,0.067458,0.932542
8,24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، المي...,1,0,0.791416,0.791416,0.208584
9,32,الممثلة أتزوج لأن الزواج منطقياً ، اذا نظرنا ب...,1,1,0.972824,0.027176,0.972824


now lets push the train probabilities and test probabilities to Vantage for processing later

In [31]:
tdml.copy_to_sql(m1_result_df, table_name="meta_learner_model1_train_results", if_exists="replace", primary_index='tid')
tdml.copy_to_sql(m1_test_result_df, table_name="meta_learner_model1_test_results", if_exists="replace", primary_index='tid')

In [35]:
# First, let's get the confusion matrix components
model1_metrics = """
SELECT 
    -- Total counts
    COUNT(*) AS total,
    SUM(CASE WHEN m1_predicted_label = 1 AND actual_label = 1 THEN 1 ELSE 0 END) AS TP,
    SUM(CASE WHEN m1_predicted_label = 0 AND actual_label = 0 THEN 1 ELSE 0 END) AS TN,
    SUM(CASE WHEN m1_predicted_label = 1 AND actual_label = 0 THEN 1 ELSE 0 END) AS FP,
    SUM(CASE WHEN m1_predicted_label = 0 AND actual_label = 1 THEN 1 ELSE 0 END) AS FN,
    SUM(CASE WHEN m1_predicted_label = actual_label THEN 1 ELSE 0 END) AS correct
FROM meta_learner_model1_test_results
"""

# Get the confusion matrix values
confusion_val = tdml.DataFrame.from_query(model1_metrics).to_pandas()

total = confusion_val['total'].values[0]
TP = confusion_val['TP'].values[0]
TN = confusion_val['TN'].values[0]
FP = confusion_val['FP'].values[0]
FN = confusion_val['FN'].values[0]
correct = confusion_val['correct'].values[0]

# Calculate metrics
accuracy = correct / total
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Display results
print("="*60)
print("MODEL 1 PERFORMANCE")
print("="*60)
print(f"\nConfusion Matrix:")
print(f"  True Positives  (TP): {TP}")
print(f"  True Negatives  (TN): {TN}")
print(f"  False Positives (FP): {FP}")
print(f"  False Negatives (FN): {FN}")
print(f"\nMetrics:")
print(f"""  Accuracy:  {accuracy*100:.2f}%""")
print(f"""  Precision: {precision*100:.2f}%""")
print(f"""  Recall:    {recall*100:.2f}%""")
print(f"""  F1-Score:  {f1_score*100:.2f}%""")
print("="*60)

MODEL 1 PERFORMANCE

Confusion Matrix:
  True Positives  (TP): 642
  True Negatives  (TN): 498
  False Positives (FP): 66
  False Negatives (FN): 70

Metrics:
  Accuracy:  89.34%
  Precision: 90.68%
  Recall:    90.17%
  F1-Score:  90.42%


In [36]:
tdml.DataFrame("meta_learner_model1_test_results").head()

tid,cleaned_doc,actual_label,m1_predicted_label,m1_predicted_label_probability,m1_prob_class_0,m1_prob_class_1
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.851382957706828,0.14861704229317196,0.851382957706828
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,0,0.5658803496671574,0.5658803496671574,0.43411965033284255
16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابتلينا المسيار يهضم حقوق المرأه,1,0,0.875465025176783,0.875465025176783,0.124534974823217
19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا الحل النهائي حو حق حقوق المرأة وبيحل جزء المشكلة، الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة جرائم الشرف منتشرة دول ماعندها ولاية اكثر بكثير اللي عندنا نظرة تشاؤمية,1,1,0.93254190873663,0.06745809126336999,0.93254190873663
27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حالياً مجلس الأمة الكويتي، مدافعة شرسة حقوق المرأة، وحق المواطن الكويتي وتمتلك عقلية اقتصادية مميزة أختيرت ضمن ٥٠ سيدة عربية مؤثرة مجتمعها,1,1,0.9882732097873426,0.011726790212657434,0.9882732097873426
29,لأ كدا الفكرة اننا بلد مابتحترم حقوق الانسان اساساً حتحترم حقوق المرأة ؟ السودان ياجماعة ماتنسو,1,1,0.5005237336935174,0.49947626630648256,0.5005237336935174
24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، الميكروفون، اللاسلكي، الراديو، السيكل، السيارة، التلفزيون، شوارع الإسفلت، الدش، الإنترنت، السينما، قيادة المرأة يقولون نقف ضد شيء لكنهم زالوا يقفون ضد المساواة الرجل المرأة نظرية التطور حقوق المثليين,1,0,0.7914156524166553,0.7914156524166553,0.2085843475833446
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.7822669045671812,0.21773309543281882,0.7822669045671812
6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.9919684812698418,0.008031518730158171,0.9919684812698418
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.9478130722349194,0.052186927765080604,0.9478130722349194


### 3c. Converting the Model 1 to ONNX

Converting the model to ONNX file only

In [37]:
onnxfile_catb = "catb-model1-approach-4.onnx"
model_catb_id = "catb-model1-approach-4"

model.save_model(onnxfile_catb,
                 format="onnx",
    export_parameters={
        'onnx_domain': 'ai.catboost',
        'onnx_model_version': 1
    })

In [38]:
import onnx
m = onnx.load('catb-model1-approach-4.onnx')
m.graph.input

[name: "features"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_param: "N"
      }
      dim {
        dim_value: 768
      }
    }
  }
}
]

In [39]:
m.graph.output

[name: "label"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_param: "N"
      }
    }
  }
}
, name: "probabilities"
type {
  sequence_type {
    elem_type {
      map_type {
        key_type: 7
        value_type {
          tensor_type {
            elem_type: 1
          }
        }
      }
    }
  }
}
]

### 3d. Loading the model 1 into Vantage

Loading the model into vantage for inference at scale

In [40]:
tdml.configure.byom_install_location = "mldb"
model_table_name = "arabic_sentiment_classifier"

try:
    tdml.db_drop_table(model_table_name)
except:
    pass

tdml.save_byom(model_id = model_catb_id, 
               model_file =  onnxfile_catb, 
               table_name =  model_table_name)

Created the model table 'arabic_sentiment_classifier' as it does not exist.
Model is saved.


In [41]:
tdml.list_byom(model_table_name)

                                          model
model_id                                       
catb-model1-approach-4  b'8031208436174426F...'


Time to infer test dataset through the classifier inside Vantage

In [42]:
df_test.shape

(1276, 771)

In [43]:
model1_onnx = tdml.retrieve_byom(model_id=model_catb_id, table_name=model_table_name)

In [44]:
onnxpred_obj = tdml.ONNXPredict(
    newdata = df_test,
    modeldata = model1_onnx,
    accumulate = ["tid", "cleaned_doc", "Sentiment"],
    overwrite_cached_models = "*",
    model_input_fields_map = f"features=[3:]"
)

print(onnxpred_obj.show_query())

SELECT * FROM "mldb".ONNXPredict(
	ON "DEMO_USER"."ml__select__1761510008346905" AS InputTable
	PARTITION BY ANY 
	ON (select model_id,model from "DEMO_USER"."ml__filter__1761518476860597") AS ModelTable
	DIMENSION
	USING
	Accumulate('tid', 'cleaned_doc', 'Sentiment')
	OverwriteCachedModel('*')
	ModelInputFieldsMap('features=[3:]')
) as sqlmr


In [45]:
onnxpred_obj.result.to_sql("model1_test_prediction_results", if_exists="replace", primary_index='tid')

In [46]:
tdml.execute_sql("SELECT TOP 5 * FROM model1_test_prediction_results;").fetchall()

[[2875,
  'المبايعة تتجدد لقيادتنا الحكيمه الشعب سلمان ومحمد الشعب',
  1,
  '{"label":[1],"probabilities":[{"info":{"size":2,"keyType":"INT64","valueType":"FLOAT"},"value":{"0":4.2676926E-5,"1":0.9999573},"type":"ONNX_TYPE_MAP"}]}'],
 [183,
  'تعدد الزوجات حال ان مقتدر ان يعدل بينهن',
  1,
  '{"label":[1],"probabilities":[{"info":{"size":2,"keyType":"INT64","valueType":"FLOAT"},"value":{"0":0.034154117,"1":0.9658459},"type":"ONNX_TYPE_MAP"}]}'],
 [2467,
  'يقولون عددنا اكثر منهم رغم انه العكس تماماً علشان يخلوننا نستسيغ فكرة تعدد الزوجات تدرون وش البلا يابنات انه ماعندهم فلوس قدره العدل واستجابة اعداد كبيره الذريه فالحين يخلفون بدون أدنى مسؤوليه وليته يتحصن همه يجمع بهالحريم ويحملهن والتربيه والمصروف زيرو',
  0,
  '{"label":[0],"probabilities":[{"info":{"size":2,"keyType":"INT64","valueType":"FLOAT"},"value":{"0":0.9794392,"1":0.020560801},"type":"ONNX_TYPE_MAP"}]}'],
 [2141,
  'سعوديات نطلب اسقاط الولايه العالم مافي امرأة عليها ولاية السفية الا المرأة السعودية حياتها يد غيرها اسقاط ال

In [47]:
try:
    tdml.db_drop_table("model1_test_set_final")
except:
    pass

query = """
CREATE TABLE model1_test_set_final AS
(
SELECT
    tid,
    cleaned_doc,
    Sentiment,
    CAST(NEW JSON(json_report).JSONExtractValue('$.label[0]') AS INTEGER) AS model1_predicted,
    CAST(NEW JSON(json_report).JSONExtractValue('$.probabilities[0].value.0') AS FLOAT) AS model1_prob_0,
    CAST(NEW JSON(json_report).JSONExtractValue('$.probabilities[0].value.1') AS FLOAT) AS model1_prob_1,
    CASE 
        WHEN CAST(NEW JSON(json_report).JSONExtractValue('$.label[0]') AS INTEGER) = 0 
        THEN CAST(NEW JSON(json_report).JSONExtractValue('$.probabilities[0].value.0') AS FLOAT)
        ELSE CAST(NEW JSON(json_report).JSONExtractValue('$.probabilities[0].value.1') AS FLOAT)
    END AS model1_predicted_probability
FROM model1_test_prediction_results
) WITH DATA
PRIMARY INDEX (tid);
"""

# Create a TeradataML DataFrame with the extracted probabilities
tdml.execute_sql(query)
result_with_probs = tdml.DataFrame("model1_test_set_final")
result_with_probs.head()

tid,cleaned_doc,Sentiment,model1_predicted,model1_prob_0,model1_prob_1,model1_predicted_probability
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.14861715,0.85138285,0.85138285
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,0,0.56588036,0.43411964,0.56588036
16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابتلينا المسيار يهضم حقوق المرأه,1,0,0.8754652,0.124534786,0.8754652
19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا الحل النهائي حو حق حقوق المرأة وبيحل جزء المشكلة، الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة جرائم الشرف منتشرة دول ماعندها ولاية اكثر بكثير اللي عندنا نظرة تشاؤمية,1,1,0.06745809,0.9325419,0.9325419
27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حالياً مجلس الأمة الكويتي، مدافعة شرسة حقوق المرأة، وحق المواطن الكويتي وتمتلك عقلية اقتصادية مميزة أختيرت ضمن ٥٠ سيدة عربية مؤثرة مجتمعها,1,1,0.011726797,0.9882732,0.9882732
29,لأ كدا الفكرة اننا بلد مابتحترم حقوق الانسان اساساً حتحترم حقوق المرأة ؟ السودان ياجماعة ماتنسو,1,1,0.4994763,0.5005237,0.5005237
24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، الميكروفون، اللاسلكي، الراديو، السيكل، السيارة، التلفزيون، شوارع الإسفلت، الدش، الإنترنت، السينما، قيادة المرأة يقولون نقف ضد شيء لكنهم زالوا يقفون ضد المساواة الرجل المرأة نظرية التطور حقوق المثليين,1,0,0.79141575,0.20858425,0.79141575
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.21773314,0.78226686,0.78226686
6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.008031547,0.99196845,0.99196845
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.052186966,0.94781303,0.94781303


lets check the accuracy of model 1

In [48]:
# First, let's get the confusion matrix components
model1_metrics = """
SELECT 
    -- Total counts
    COUNT(*) AS total,
    SUM(CASE WHEN model1_predicted = 1 AND Sentiment = 1 THEN 1 ELSE 0 END) AS TP,
    SUM(CASE WHEN model1_predicted = 0 AND Sentiment = 0 THEN 1 ELSE 0 END) AS TN,
    SUM(CASE WHEN model1_predicted = 1 AND Sentiment = 0 THEN 1 ELSE 0 END) AS FP,
    SUM(CASE WHEN model1_predicted = 0 AND Sentiment = 1 THEN 1 ELSE 0 END) AS FN,
    SUM(CASE WHEN model1_predicted = Sentiment THEN 1 ELSE 0 END) AS correct
FROM model1_test_set_final
"""

# Get the confusion matrix values
confusion_val = tdml.DataFrame.from_query(model1_metrics).to_pandas()

total = confusion_val['total'].values[0]
TP = confusion_val['TP'].values[0]
TN = confusion_val['TN'].values[0]
FP = confusion_val['FP'].values[0]
FN = confusion_val['FN'].values[0]
correct = confusion_val['correct'].values[0]

# Calculate metrics
accuracy = correct / total
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Display results
print("="*60)
print("MODEL 1 PERFORMANCE")
print("="*60)
print(f"\nConfusion Matrix:")
print(f"  True Positives  (TP): {TP}")
print(f"  True Negatives  (TN): {TN}")
print(f"  False Positives (FP): {FP}")
print(f"  False Negatives (FN): {FN}")
print(f"\nMetrics:")
print(f"""  Accuracy:  {accuracy*100:.2f}%""")
print(f"""  Precision: {precision*100:.2f}%""")
print(f"""  Recall:    {recall*100:.2f}%""")
print(f"""  F1-Score:  {f1_score*100:.2f}%""")
print("="*60)

MODEL 1 PERFORMANCE

Confusion Matrix:
  True Positives  (TP): 642
  True Negatives  (TN): 498
  False Positives (FP): 66
  False Negatives (FN): 70

Metrics:
  Accuracy:  89.34%
  Precision: 90.68%
  Recall:    90.17%
  F1-Score:  90.42%


## 4. Creating the 2nd model with CountVectorizer and LR Classifier

This model will be utilizing the SKlearn countvectorizer() coupled with scaling and Random Forest for classification

### 4a. Loading the cleaned dataset without embeddings

Loading the cleaned dataset for vectorization

In [49]:
cleaned_df = tdml.DataFrame("sentiment_analysis_data_cleaned")
cleaned_df.head()

tid,cleaned_doc,Sentiment
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1
4,ولي امر الزوجة او ولي الزوجة او ولي المراة الاخطاء الشائعة الكلام غلط الشريعة والقانون يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها الخ وليس الزوج منهم حقوق عليها لكنها ولاية الولاية مصطلح فقهي قانوني محدد,1
5,انا كامرأة يوجعني حزن نساء العالم، سواء تعنفت، ضُربت، قُتلت جريمة شرف، اغتُصِبت، حرِمت دراسة عمل جبرت زواج او حياة تخترتها الدفاع حقوق المرأة الي تُعتبر أغلب دول العالم مواطن بالدرجة الثانية، مو ترند لكنه إنسانية والساكت الحق شيطان أخرس,1
6,اومن حقوق المرأة أنك تفتحلها الباب,1
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1
9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق المرأة ترى قانون او عقوبه الزوج الي بيمارس العنف ضد زوجته الشارع ؟؟؟,1
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1
3,حقوق المرأة تضمنها وزارة العدل,1
1,حقوق المرأة الإسلام,1
0,حقوق المرأة,1


In [50]:
TrainTestSplit_out_2 = tdml.TrainTestSplit(
                                    data = cleaned_df,
                                    id_column = "tid",
                                    train_size = 0.70,
                                    test_size = 0.30,
                                    seed = 21
)

# Split into 2 virtual dataframes
cleaned_df_train = TrainTestSplit_out_2.result[TrainTestSplit_out_2.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
cleaned_df_test = TrainTestSplit_out_2.result[TrainTestSplit_out_2.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

In [51]:
cleaned_df_train.to_csv("trainset_model2.csv", all_rows=True, fastexport=False)
trainingdataset2 = pd.read_csv("trainset_model2.csv")
trainingdataset2.head()


Data is successfully exported into trainset_model2.csv


,tid,cleaned_doc,Sentiment
0,1,حقوق المرأة الإسلام,1
1,9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق ا...,1
2,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1
3,0,حقوق المرأة,1
4,3,حقوق المرأة تضمنها وزارة العدل,1


### 4b. Training the SKlearn pipeline on the Train Set

Time to train the 2nd model using 5-fold CV

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

lr_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(
        max_features=10000,
        stop_words=arabic_stopwords,
        ngram_range=(1, 2)
    )),
    ('scaler', StandardScaler(with_mean=False)),  # with_mean=False for sparse matrices
    ('classifier', LogisticRegression(
        C=1.0,                      # Regularization strength (smaller = stronger regularization)
        penalty='l2',               # L2 regularization (Ridge)
        solver='saga',              # Best for large datasets, supports L1/L2/ElasticNet
        max_iter=1000,              # Increase if convergence warning appears
        class_weight='balanced',    # Handle class imbalance
        random_state=42,
        n_jobs=-1
    ))
])

In [53]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

print(f"Training set shape: {trainingdataset2.shape}")
print(f"Columns: {trainingdataset2.columns.tolist()[:10]}...")  # Show first 10 columns

# ============================================================================
# STEP 3: Prepare features and labels
# ============================================================================
# Extract row IDs, text, and labels
train_ids = trainingdataset2['tid'].values
train_text = trainingdataset2['cleaned_doc'].values
y_train = trainingdataset2['Sentiment'].values
X_train = train_text  # For text data, features are the raw text

print(f"\nTraining set size: {len(X_train)}")
print(f"Class distribution: {np.bincount(y_train)}")

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize arrays to store out-of-fold predictions
oof_probs = np.zeros((len(X_train), 2))  # Float is fine for probabilities
oof_preds = np.zeros(len(X_train), dtype=int)  # ← Specify dtype=int for predictions!

print(f"\nStarting {n_folds}-fold cross-validation...")
print("="*70)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"\n{'='*70}")
    print(f"FOLD {fold}/{n_folds}")
    print(f"{'='*70}")
    print(f"Train size: {len(train_idx)}, Validation size: {len(val_idx)}")
    
    # Split data for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    
    # Train the model on this fold
    print(f"Training model...")
    lr_pipeline.fit(X_fold_train, y_fold_train)
    
    # Get predictions on validation fold
    oof_probs[val_idx] = lr_pipeline.predict_proba(X_fold_val)
    oof_preds[val_idx] = lr_pipeline.predict(X_fold_val)

    # Calculate fold metrics
    fold_acc = accuracy_score(y_fold_val, oof_preds[val_idx])
    fold_f1 = f1_score(y_fold_val, oof_preds[val_idx])
    fold_auc = roc_auc_score(y_fold_val, oof_probs[val_idx, 1])
    
    print(f"Fold {fold} - Accuracy: {fold_acc:.4f}, F1: {fold_f1:.4f}, AUC: {fold_auc:.4f}")

print("\n" + "="*70)
print("Cross-validation complete!")
print("="*70)


Training set shape: (2975, 3)
Columns: ['tid', 'cleaned_doc', 'Sentiment']...

Training set size: 2975
Class distribution: [1251 1724]

Starting 5-fold cross-validation...

FOLD 1/5
Train size: 2380, Validation size: 595
Training model...


/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(
/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fold 1 - Accuracy: 0.8370, F1: 0.8680, AUC: 0.9280

FOLD 2/5
Train size: 2380, Validation size: 595
Training model...


/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fold 2 - Accuracy: 0.8571, F1: 0.8818, AUC: 0.9302

FOLD 3/5
Train size: 2380, Validation size: 595
Training model...


/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fold 3 - Accuracy: 0.8588, F1: 0.8810, AUC: 0.9280

FOLD 4/5
Train size: 2380, Validation size: 595
Training model...


/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Fold 4 - Accuracy: 0.8706, F1: 0.8947, AUC: 0.9567

FOLD 5/5
Train size: 2380, Validation size: 595
Training model...
Fold 5 - Accuracy: 0.8706, F1: 0.8902, AUC: 0.9456

Cross-validation complete!


/Users/huzaifahsaleem/anaconda3/envs/arabic_sentiment/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [54]:
oof_acc = accuracy_score(y_train, oof_preds)
oof_f1 = f1_score(y_train, oof_preds)
oof_auc = roc_auc_score(y_train, oof_probs[:, 1])

print("\n" + "="*70)
print("OVERALL OUT-OF-FOLD PERFORMANCE")
print("="*70)
print(f"Accuracy:  {oof_acc:.4f}")
print(f"F1-Score:  {oof_f1:.4f}")
print(f"AUC:       {oof_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_train, oof_preds))

predicted_label_prob = np.where(
    oof_preds == 1, 
    oof_probs[:, 1],  # If predicted 1, take prob of class 1
    oof_probs[:, 0]   # If predicted 0, take prob of class 0
)

# Create the final DataFrame
m2_result_df = pd.DataFrame({
    'tid': train_ids,
    'cleaned_doc': train_text,
    'actual_label': y_train,
    'm2_predicted_label': oof_preds.astype(int),
    'm2_predicted_label_probability': predicted_label_prob,
    'm2_prob_class_0': oof_probs[:, 0],
    'm2_prob_class_1': oof_probs[:, 1]
})

print("\n" + "="*70)
print("OUT-OF-FOLD PREDICTIONS DATAFRAME")
print("="*70)
print(f"Shape: {m2_result_df.shape}")
print(f"\nColumns: {list(m2_result_df.columns)}")
print("\nFirst 10 rows:")
m2_result_df.head(10)


OVERALL OUT-OF-FOLD PERFORMANCE
Accuracy:  0.8588
F1-Score:  0.8831
AUC:       0.9374

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.77      0.82      1251
           1       0.85      0.92      0.88      1724

    accuracy                           0.86      2975
   macro avg       0.86      0.85      0.85      2975
weighted avg       0.86      0.86      0.86      2975


OUT-OF-FOLD PREDICTIONS DATAFRAME
Shape: (2975, 7)

Columns: ['tid', 'cleaned_doc', 'actual_label', 'm2_predicted_label', 'm2_predicted_label_probability', 'm2_prob_class_0', 'm2_prob_class_1']

First 10 rows:


,tid,cleaned_doc,actual_label,m2_predicted_label,m2_predicted_label_probability,m2_prob_class_0,m2_prob_class_1
0,1,حقوق المرأة الإسلام,1,1,0.550173,0.449827,0.550173
1,9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق ا...,1,1,0.991993,0.008007,0.991993
2,4,ولي امر الزوجة او ولي الزوجة او ولي المراة الا...,1,1,0.953926,0.046074,0.953926
3,0,حقوق المرأة,1,1,0.662008,0.337992,0.662008
4,3,حقوق المرأة تضمنها وزارة العدل,1,1,0.808802,0.191198,0.808802
5,11,محاولات بائسه لاقصاء دور الام السعودية ودورها ...,1,0,0.542471,0.542471,0.457529
6,12,كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، فالل...,1,1,0.950059,0.049941,0.950059
7,14,تعرف ان المرأة عندها حقوق؟ يوم يطبعون ورقة,1,1,0.924499,0.075501,0.924499
8,5,انا كامرأة يوجعني حزن نساء العالم، سواء تعنفت،...,1,1,0.996888,0.003112,0.996888
9,13,الحقوق او المزعوم انها حقوق عبارة صفر مكعب مال...,1,1,0.940763,0.059237,0.940763


In [55]:
tdml.copy_to_sql(m2_result_df, table_name="meta_learner_model2_train_results", if_exists="replace", primary_index='tid')

In [56]:
tdml.execute_sql("SELECT TOP 1 * FROM meta_learner_model2_train_results;").fetchall()

[[1876,
  'ولي العهد يدعم ابناء الوطن رؤية تسعى الى توطين السكك الحديدة لذلك زيارة ولي العهد تاتي اجل دعمه',
  1,
  1,
  0.9918491248637035,
  0.008150875136296531,
  0.9918491248637035]]

### 4c. Evaluating the Model 2 on Test Set

In [57]:
test_dataset2 = cleaned_df_test.to_csv("testset_model2.csv", all_rows=True, fastexport=False)
test_dataset2 = pd.read_csv("testset_model2.csv")
test_dataset2.head()


Data is successfully exported into testset_model2.csv


,tid,cleaned_doc,Sentiment
0,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1
1,10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وه...,1
2,7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المت...,1
3,6,اومن حقوق المرأة أنك تفتحلها الباب,1
4,8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1


In [60]:
print(f"Test set shape: {test_dataset2.shape}")

# Extract test IDs, text, and labels
test_ids = test_dataset2['tid'].values
test_text = test_dataset2['cleaned_doc'].values
y_test = test_dataset2['Sentiment'].values

# Extract embedding features (same columns as training)
X_test = test_text # For text data, features are the raw text

print(f"Test set size: {len(X_test)}")
print(f"Class distribution: {np.bincount(y_test)}")

# ============================================================================
# STEP 3: Make predictions on test set
# ============================================================================
print("\n" + "="*70)
print("MAKING PREDICTIONS ON TEST SET")
print("="*70)

# Get predictions and probabilities
test_probs = lr_pipeline.predict_proba(X_test)
test_preds = lr_pipeline.predict(X_test)

print(f"✓ Generated predictions for {len(test_preds)} samples")

# ============================================================================
# STEP 4: Calculate test set performance
# ============================================================================
test_acc = accuracy_score(y_test, test_preds)
test_f1 = f1_score(y_test, test_preds)
test_auc = roc_auc_score(y_test, test_probs[:, 1])

print("\n" + "="*70)
print("TEST SET PERFORMANCE")
print("="*70)
print(f"Accuracy:  {test_acc:.4f}")
print(f"F1-Score:  {test_f1:.4f}")
print(f"AUC:       {test_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, test_preds))

print("\nConfusion Matrix:")
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, test_preds)
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives:  {cm[1,1]}")

# ============================================================================
# STEP 5: Create DataFrame with test predictions and probabilities
# ============================================================================
# For each row, store the probability of the PREDICTED label
predicted_label_prob = np.where(
    test_preds == 1, 
    test_probs[:, 1],  # If predicted 1, take prob of class 1
    test_probs[:, 0]   # If predicted 0, take prob of class 0
)

# Create the test results DataFrame
m2_test_result_df = pd.DataFrame({
    'tid': test_ids,
    'cleaned_doc': test_text,
    'actual_label': y_test,
    'm2_predicted_label': test_preds.astype(int),
    'm2_predicted_label_probability': predicted_label_prob,
    # Include both probabilities for meta-learner
    'm2_prob_class_0': test_probs[:, 0],
    'm2_prob_class_1': test_probs[:, 1]
})

print("\n" + "="*70)
print("TEST SET PREDICTIONS DATAFRAME")
print("="*70)
print(f"Shape: {m2_test_result_df.shape}")
print(f"\nColumns: {list(m2_test_result_df.columns)}")
print("\nFirst 10 rows:")
m2_test_result_df.head(10)


Test set shape: (1276, 3)
Test set size: 1276
Class distribution: [564 712]

MAKING PREDICTIONS ON TEST SET
✓ Generated predictions for 1276 samples

TEST SET PERFORMANCE
Accuracy:  0.8597
F1-Score:  0.8765
AUC:       0.9371

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.82      0.84       564
           1       0.86      0.89      0.88       712

    accuracy                           0.86      1276
   macro avg       0.86      0.86      0.86      1276
weighted avg       0.86      0.86      0.86      1276


Confusion Matrix:
[[462 102]
 [ 77 635]]

True Negatives:  462
False Positives: 102
False Negatives: 77
True Positives:  635

TEST SET PREDICTIONS DATAFRAME
Shape: (1276, 7)

Columns: ['tid', 'cleaned_doc', 'actual_label', 'm2_predicted_label', 'm2_predicted_label_probability', 'm2_prob_class_0', 'm2_prob_class_1']

First 10 rows:


,tid,cleaned_doc,actual_label,m2_predicted_label,m2_predicted_label_probability,m2_prob_class_0,m2_prob_class_1
0,2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق...,1,1,0.566748,0.433252,0.566748
1,10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وه...,1,1,0.974155,0.025845,0.974155
2,7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المت...,1,1,0.801440,0.198560,0.801440
3,6,اومن حقوق المرأة أنك تفتحلها الباب,1,0,0.734721,0.734721,0.265279
4,8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.877210,0.122790,0.877210
5,16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابت...,1,1,0.894660,0.105340,0.894660
6,27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حال...,1,1,0.962313,0.037687,0.962313
7,19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا ال...,1,1,0.998683,0.001317,0.998683
8,24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، المي...,1,0,0.675816,0.675816,0.324184
9,32,الممثلة أتزوج لأن الزواج منطقياً ، اذا نظرنا ب...,1,1,0.867913,0.132087,0.867913


### 4c. Converting the pipeline (LR) to ONNX

In [61]:
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
import onnxruntime as ort

# Convert to ONNX
print("\nConverting to ONNX...")
initial_type = [('input', StringTensorType([None, 1]))]

# Define separators for tokenization
seps = {
    CountVectorizer: {
        "separators": [' ', '[.]', '\\?', ',', ';', ':', '!', 
                      '\\(', '\\)', '\n', '\\"', "'", "-", 
                      "\\[", "\\]", "@"]
    }
}

# Convert to ONNX
onnx_model = convert_sklearn(
    lr_pipeline,
    initial_types=initial_type,
    options=seps,
    target_opset=16,
    dtype=np.float32
)

# Save ONNX model
output_path = "lr-model2-approach4.onnx"
with open(output_path, "wb") as f:
    f.write(onnx_model.SerializeToString())
    
print(f"ONNX model saved to: {output_path}")

# Test ONNX model locally
import onnxruntime as ort

session = ort.InferenceSession(output_path)
input_name = session.get_inputs()[0].name
output_names = [output.name for output in session.get_outputs()]

# Test on a few samples
test_texts = X_test[:5]
test_array = np.array(test_texts).reshape(-1, 1)

onnx_results = session.run(output_names, {input_name: test_array})

print("\n" + "="*60)
print("ONNX Model Test Results:")
print("="*60)
for i, text in enumerate(test_texts):
    print(f"\nText: {text[:80]}")
    print(f"  Prediction: {onnx_results[0][i]}")
    print(f"  Probabilities: {onnx_results[1][i]}")


Converting to ONNX...
ONNX model saved to: lr-model2-approach4.onnx

ONNX Model Test Results:

Text: لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك
  Prediction: 1
  Probabilities: {0: 0.4332521855831146, 1: 0.566747784614563}

Text: المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو 
  Prediction: 1
  Probabilities: {0: 0.025845319032669067, 1: 0.9741547107696533}

Text: غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك 
  Prediction: 1
  Probabilities: {0: 0.19855999946594238, 1: 0.8014400005340576}

Text: اومن حقوق المرأة أنك تفتحلها الباب
  Prediction: 0
  Probabilities: {0: 0.7347208261489868, 1: 0.2652791738510132}

Text: مبدائيا انا حقوق المرأة وضد حقوق الرجل
  Prediction: 1
  Probabilities: {0: 0.12278994917869568, 1: 0.8772100210189819}


### 4d. Loading the model 2 in Vantage for inferencing at scale

In [62]:
model2_file = os.path.join("lr-model2-approach4.onnx")
model2_id = "lr-model2-approach4"
model_table_name = "arabic_sentiment_classifier"

In [63]:
tdml.save_byom(model_id=model2_id, model_file=model2_file, table_name=model_table_name)

Model is saved.


In [64]:
tdml.list_byom(model_table_name)

                                          model
model_id                                       
catb-model1-approach-4  b'8031208436174426F...'
lr-model2-approach4     b'8081208736B6C326F...'


In [65]:
model2_onnx = tdml.retrieve_byom(model_id=model2_id, table_name=model_table_name)
model2_onnx

model_id,model
lr-model2-approach4,b'8081208736B6C326F...'


In [66]:
model2_result = tdml.ONNXPredict(accumulate=["tid", "cleaned_doc", "Sentiment"],
                                   overwrite_cached_models="*",
                                   model_input_fields_map= 'input = cleaned_doc',
                                   newdata=cleaned_df_test,
                                   modeldata=model2_onnx)

In [67]:
tdml.db_list_tables()

,TableName
0,jcr_sentiment_arabic_inference
1,ml___frmqry_v_1761492939325222
2,ml__td_sqlmr_out__1761511500915266
3,ml__filter__1761518476860597
4,ml__select__1761513260184541
5,ml___frmqry_v_1761511991257212
6,ml__select__1761510008346905
7,ml___frmqry_v_1761493670463464
8,ml__filter__1761512417999147
9,ml__td_sqlmr_out__1761494497473059


In [68]:
model2_result.result.to_sql("model2_test_prediction_results", if_exists="replace", primary_index='tid')

In [69]:
tdml.execute_sql("select TOP 1 * from model2_test_prediction_results;").fetchall()

[[469,
  'الولاية عبودية ضد المرأة استغلها ذكور للتسلط عليها سعوديات نطلب اسقاط الولايه',
  1,
  '{"output_probability":[{"info":{"size":2,"keyType":"INT64","valueType":"FLOAT"},"value":{"0":0.08993095,"1":0.91006905},"type":"ONNX_TYPE_MAP"}],"output_label":[1]}']]

In [70]:
try:
    tdml.db_drop_table("model2_test_set_final")
except:
    pass

query = """
CREATE TABLE model2_test_set_final AS (
SELECT
    tid,
    cleaned_doc,
    Sentiment,
    CAST(NEW JSON(json_report).JSONExtractValue('$.output_label[0]') AS INTEGER) AS model2_predicted,
        CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.0') AS FLOAT) AS model2_prob_0,
    CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.1') AS FLOAT) AS model2_prob_1,
    CASE 
        WHEN CAST(NEW JSON(json_report).JSONExtractValue('$.output_label[0]') AS INTEGER) = 0 
        THEN CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.0') AS FLOAT)
        ELSE CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.1') AS FLOAT)
    END AS model2_predicted_probability
FROM model2_test_prediction_results
) WITH DATA
PRIMARY INDEX (tid);
"""

# Create a TeradataML DataFrame with the extracted probabilities
tdml.execute_sql(query)
result_with_probs_2 = tdml.DataFrame("model2_test_set_final")
result_with_probs_2.head()

tid,cleaned_doc,Sentiment,model2_predicted,model2_prob_0,model2_prob_1,model2_predicted_probability
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.19856003,0.80144,0.80144
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,1,0.025845349,0.97415465,0.97415465
16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابتلينا المسيار يهضم حقوق المرأه,1,1,0.105339885,0.8946601,0.8946601
19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا الحل النهائي حو حق حقوق المرأة وبيحل جزء المشكلة، الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة جرائم الشرف منتشرة دول ماعندها ولاية اكثر بكثير اللي عندنا نظرة تشاؤمية,1,1,0.0052167475,0.9947833,0.9947833
27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حالياً مجلس الأمة الكويتي، مدافعة شرسة حقوق المرأة، وحق المواطن الكويتي وتمتلك عقلية اقتصادية مميزة أختيرت ضمن ٥٠ سيدة عربية مؤثرة مجتمعها,1,1,0.071546614,0.9284534,0.9284534
29,لأ كدا الفكرة اننا بلد مابتحترم حقوق الانسان اساساً حتحترم حقوق المرأة ؟ السودان ياجماعة ماتنسو,1,1,0.08163872,0.9183613,0.9183613
24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، الميكروفون، اللاسلكي، الراديو، السيكل، السيارة، التلفزيون، شوارع الإسفلت، الدش، الإنترنت، السينما، قيادة المرأة يقولون نقف ضد شيء لكنهم زالوا يقفون ضد المساواة الرجل المرأة نظرية التطور حقوق المثليين,1,0,0.6839293,0.3160707,0.6839293
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.12278998,0.87721,0.87721
6,اومن حقوق المرأة أنك تفتحلها الباب,1,0,0.7347209,0.2652791,0.7347209
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.43325222,0.5667478,0.5667478


Lets check the accuracy of model 2

In [71]:
# First, let's get the confusion matrix components
model2_metrics = """
SELECT 
    -- Total counts
    COUNT(*) AS total,
    SUM(CASE WHEN model2_predicted = 1 AND Sentiment = 1 THEN 1 ELSE 0 END) AS TP,
    SUM(CASE WHEN model2_predicted = 0 AND Sentiment = 0 THEN 1 ELSE 0 END) AS TN,
    SUM(CASE WHEN model2_predicted = 1 AND Sentiment = 0 THEN 1 ELSE 0 END) AS FP,
    SUM(CASE WHEN model2_predicted = 0 AND Sentiment = 1 THEN 1 ELSE 0 END) AS FN,
    SUM(CASE WHEN model2_predicted = Sentiment THEN 1 ELSE 0 END) AS correct
FROM model2_test_set_final
"""

# Get the confusion matrix values
confusion_val = tdml.DataFrame.from_query(model2_metrics).to_pandas()

total = confusion_val['total'].values[0]
TP = confusion_val['TP'].values[0]
TN = confusion_val['TN'].values[0]
FP = confusion_val['FP'].values[0]
FN = confusion_val['FN'].values[0]
correct = confusion_val['correct'].values[0]

# Calculate metrics
accuracy = correct / total
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Display results
print("="*60)
print("MODEL 2 PERFORMANCE")
print("="*60)
print(f"\nConfusion Matrix:")
print(f"  True Positives  (TP): {TP}")
print(f"  True Negatives  (TN): {TN}")
print(f"  False Positives (FP): {FP}")
print(f"  False Negatives (FN): {FN}")
print(f"\nMetrics:")
print(f"""  Accuracy:  {accuracy*100:.2f}%""")
print(f"""  Precision: {precision*100:.2f}%""")
print(f"""  Recall:    {recall*100:.2f}%""")
print(f"""  F1-Score:  {f1_score*100:.2f}%""")
print("="*60)

MODEL 2 PERFORMANCE

Confusion Matrix:
  True Positives  (TP): 634
  True Negatives  (TN): 460
  False Positives (FP): 104
  False Negatives (FN): 78

Metrics:
  Accuracy:  85.74%
  Precision: 85.91%
  Recall:    89.04%
  F1-Score:  87.45%


## 5. Merging the train set and test set from both models for the Meta Learner

In [72]:
m1_train = tdml.DataFrame("meta_learner_model1_train_results")
m2_train = tdml.DataFrame("meta_learner_model2_train_results")

In [73]:
m2_train.head()

tid,cleaned_doc,actual_label,m2_predicted_label,m2_predicted_label_probability,m2_prob_class_0,m2_prob_class_1
3,حقوق المرأة تضمنها وزارة العدل,1,1,0.8088022917343768,0.1911977082656232,0.8088022917343768
5,انا كامرأة يوجعني حزن نساء العالم، سواء تعنفت، ضُربت، قُتلت جريمة شرف، اغتُصِبت، حرِمت دراسة عمل جبرت زواج او حياة تخترتها الدفاع حقوق المرأة الي تُعتبر أغلب دول العالم مواطن بالدرجة الثانية، مو ترند لكنه إنسانية والساكت الحق شيطان أخرس,1,1,0.9968883983738157,0.0031116016261842905,0.9968883983738157
9,السؤال بقى للناس الي شغاله حقوق الانسان حقوق المرأة ترى قانون او عقوبه الزوج الي بيمارس العنف ضد زوجته الشارع ؟؟؟,1,1,0.9919925931549094,0.008007406845090603,0.9919925931549094
11,محاولات بائسه لاقصاء دور الام السعودية ودورها التربية وغرس الانتماء بفلذات اكبادها لذالك شي حق للذكور فقط رؤية ستنصف الام السعودية وستدمر الافكار الذكورية الجاهلية تجنيس ابناء السعوديات اولويات حقوق المراة السعودية,1,0,0.5424714051333306,0.5424714051333306,0.45752859486666936
13,الحقوق او المزعوم انها حقوق عبارة صفر مكعب مالم تسقط الولاية المرأة,1,1,0.9407628279233462,0.0592371720766538,0.9407628279233462
14,تعرف ان المرأة عندها حقوق؟ يوم يطبعون ورقة,1,1,0.9244990235877787,0.07550097641222131,0.9244990235877787
12,كانت مسألة حقوق المرأة أمرا سخيفاً جداً ، فاللوم يقع غطرسة الرجال جعلت موضوعًا يُناقش,1,1,0.950058627993368,0.04994137200663196,0.950058627993368
4,ولي امر الزوجة او ولي الزوجة او ولي المراة الاخطاء الشائعة الكلام غلط الشريعة والقانون يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها الخ وليس الزوج منهم حقوق عليها لكنها ولاية الولاية مصطلح فقهي قانوني محدد,1,1,0.9539259249250355,0.04607407507496453,0.9539259249250355
1,حقوق المرأة الإسلام,1,1,0.5501734652697549,0.44982653473024514,0.5501734652697549
0,حقوق المرأة,1,1,0.662008128437408,0.33799187156259203,0.662008128437408


Train data

In [74]:
# Saving the results as a permanent table in Teradata

# First, drop the table if it exists
try:
    tdml.db_drop_table("ensemble_train_data")
except:
    pass

# Create table with extracted probabilities from both models
create_query = """
CREATE TABLE ensemble_train_data AS (
    SELECT 
        m1.tid,
        m1.cleaned_doc,
        m1.actual_label as Sentiment,
        m1.m1_predicted_label AS model1_predicted,
        m1.m1_predicted_label_probability AS model1_predicted_probability,
        m1.m1_prob_class_0 AS model1_prob_0,
        m1.m1_prob_class_1 AS model1_prob_1,
        m2.m2_predicted_label AS model2_predicted,
        m2.m2_predicted_label_probability AS model2_predicted_probability,
        m2.m2_prob_class_0 AS model2_prob_0,
        m2.m2_prob_class_1 AS model2_prob_1
    FROM meta_learner_model1_train_results m1
    INNER JOIN meta_learner_model2_train_results m2
        ON m1.tid = m2.tid
) WITH DATA
PRIMARY INDEX (tid);
"""

# Execute the CREATE TABLE query
tdml.execute_sql(create_query)

# Load the table as a TeradataML DataFrame
full_train_data = tdml.DataFrame("ensemble_train_data")

# Display the result
print("Combined train data from both models saved to 'ensemble_train_data' table:")
print(f"Shape: {full_train_data.shape}")
full_train_data.head(5)

Combined train data from both models saved to 'ensemble_train_data' table:
Shape: (2975, 11)


tid,cleaned_doc,Sentiment,model1_predicted,model1_predicted_probability,model1_prob_0,model1_prob_1,model2_predicted,model2_predicted_probability,model2_prob_0,model2_prob_1
3,حقوق المرأة تضمنها وزارة العدل,1,1,0.992387959632813,0.00761204036718699,0.992387959632813,1,0.8088022917343768,0.1911977082656232,0.8088022917343768
5,انا كامرأة يوجعني حزن نساء العالم، سواء تعنفت، ضُربت، قُتلت جريمة شرف، اغتُصِبت، حرِمت دراسة عمل جبرت زواج او حياة تخترتها الدفاع حقوق المرأة الي تُعتبر أغلب دول العالم مواطن بالدرجة الثانية، مو ترند لكنه إنسانية والساكت الحق شيطان أخرس,1,1,0.9953827964505518,0.004617203549448168,0.9953827964505518,1,0.9968883983738157,0.0031116016261842905,0.9968883983738157
4,ولي امر الزوجة او ولي الزوجة او ولي المراة الاخطاء الشائعة الكلام غلط الشريعة والقانون يوجد ولي للزوجة او المراة الا اذا كانت قاصرا ويكون الولي ابوها الخ وليس الزوج منهم حقوق عليها لكنها ولاية الولاية مصطلح فقهي قانوني محدد,1,1,0.9808368869216572,0.01916311307834284,0.9808368869216572,1,0.9539259249250355,0.04607407507496453,0.9539259249250355
1,حقوق المرأة الإسلام,1,1,0.9962086894027884,0.0037913105972116057,0.9962086894027884,1,0.5501734652697549,0.44982653473024514,0.5501734652697549
0,حقوق المرأة,1,1,0.9944215166146609,0.005578483385339061,0.9944215166146609,1,0.662008128437408,0.33799187156259203,0.662008128437408


Test data

In [75]:
# Saving the results as a permanent table in Teradata

# First, drop the table if it exists
try:
    tdml.db_drop_table("ensemble_test_data")
except:
    pass

# Create table with extracted probabilities from both models
create_query = """
CREATE TABLE ensemble_test_data AS (
    SELECT 
        m1.tid,
        m1.cleaned_doc,
        m1.Sentiment,
        m1.model1_predicted,
        m1.model1_predicted_probability,
        m1.model1_prob_0,
        m1.model1_prob_1,
        m2.model2_predicted,
        m2.model2_predicted_probability,
        m2.model2_prob_0,
        m2.model2_prob_1
    FROM model1_test_set_final m1
    INNER JOIN model2_test_set_final m2
        ON m1.tid = m2.tid
) WITH DATA
PRIMARY INDEX (tid);
"""

# Execute the CREATE TABLE query
tdml.execute_sql(create_query)

# Load the table as a TeradataML DataFrame
full_test_data = tdml.DataFrame("ensemble_test_data")

# Display the result
print("Combined test data from both models saved to 'ensemble_test_data' table:")
print(f"Shape: {full_test_data.shape}")
full_test_data.head(5)

Combined test data from both models saved to 'ensemble_test_data' table:
Shape: (1276, 11)


tid,cleaned_doc,Sentiment,model1_predicted,model1_predicted_probability,model1_prob_0,model1_prob_1,model2_predicted,model2_predicted_probability,model2_prob_0,model2_prob_1
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.85138285,0.14861715,0.85138285,1,0.80144,0.19856003,0.80144
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,0,0.56588036,0.56588036,0.43411964,1,0.97415465,0.025845349,0.97415465
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.78226686,0.21773314,0.78226686,1,0.87721,0.12278998,0.87721
6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.99196845,0.008031547,0.99196845,0,0.7347209,0.7347209,0.2652791
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.94781303,0.052186966,0.94781303,1,0.5667478,0.43325222,0.5667478


## 6. Time to Train & Evaluate a Meta Learner

Now lets do the easy work and train the meta learner. For this, lets go with a simple one. Logistic Regression.

In [76]:
full_train_data.to_csv("ensemble_train_data.csv", all_rows=True, fastexport=False)
ensemble_training_data = pd.read_csv("ensemble_train_data.csv")
ensemble_training_data.head()


Data is successfully exported into ensemble_train_data.csv


,tid,cleaned_doc,Sentiment,model1_predicted,model1_predicted_probability,model1_prob_0,model1_prob_1,model2_predicted,model2_predicted_probability,model2_prob_0,model2_prob_1
0,3752,قيادة المرأة للسيارة رسميا اليوم زوجتي الله يح...,1,1,0.998084,0.001916,0.998084,1,0.904753,0.095247,0.904753
1,1876,ولي العهد يدعم ابناء الوطن رؤية تسعى الى توطين...,1,1,0.999933,0.000067,0.999933,1,0.991849,0.008151,0.991849
2,3487,عطاله التخصصات الصحيه١ ادخل الثلاثين وانا عاطل...,0,0,0.999973,0.999973,0.000027,0,0.996698,0.996698,0.003302
3,0,حقوق المرأة,1,1,0.994422,0.005578,0.994422,1,0.662008,0.337992,0.662008
4,2610,شعار الجزيرة الراي والراي الاخر استضافت مدافعي...,0,0,0.999870,0.999870,0.000130,0,0.864007,0.864007,0.135993


In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd

feature_cols_a = [
    'model1_predicted', 
    'model1_predicted_probability',
    'model1_prob_0',
    'model1_prob_1',
    'model2_predicted',
    'model2_predicted_probability', 
    'model2_prob_0',
    'model2_prob_1'
]

# Option B: Use only the 4 probabilities (simpler, often works well)
feature_cols_b = [
     'model1_prob_0',
     'model1_prob_1',
     'model2_prob_0',
     'model2_prob_1'
 ]

# Option C: Use only predicted probabilities (simplest)
feature_cols_c = [
     'model1_predicted_probability',
     'model2_predicted_probability'
 ]

X_meta_train = ensemble_training_data[feature_cols_a].values
y_meta_train = ensemble_training_data['Sentiment'].values


meta_learner = LogisticRegression(
    C=1.0,                      # Regularization strength
    penalty='l2',               # L2 regularization
    max_iter=1000,
    random_state=42
)


""" ## Tried catboost - 90.04% accuracy
meta_learner = catboost.CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    early_stopping_rounds=50,
    verbose=False,
    random_state=42
)
"""

meta_learner.fit(X_meta_train, y_meta_train)
print("Meta-learner trained on ensemble features.")

y_meta_train_pred = meta_learner.predict(X_meta_train)
y_meta_train_proba = meta_learner.predict_proba(X_meta_train)

train_acc = accuracy_score(y_meta_train, y_meta_train_pred)
train_f1 = f1_score(y_meta_train, y_meta_train_pred)
train_auc = roc_auc_score(y_meta_train, y_meta_train_proba[:, 1])

print(f"Training Accuracy:  {train_acc:.4f}")
print(f"Training F1-Score:  {train_f1:.4f}")
print(f"Training AUC:       {train_auc:.4f}")

Meta-learner trained on ensemble features.
Training Accuracy:  0.9150
Training F1-Score:  0.9273
Training AUC:       0.9732


lets evaluate the model on test set before conversion to ONNX

In [78]:
full_test_data.to_csv("ensemble_test_data.csv", all_rows=True, fastexport=False)
ensemble_testing_data = pd.read_csv("ensemble_test_data.csv")
ensemble_testing_data.head()


Data is successfully exported into ensemble_test_data.csv


,tid,cleaned_doc,Sentiment,model1_predicted,model1_predicted_probability,model1_prob_0,model1_prob_1,model2_predicted,model2_predicted_probability,model2_prob_0,model2_prob_1
0,469,الولاية عبودية ضد المرأة استغلها ذكور للتسلط ع...,1,1,0.999546,0.000454,0.999546,1,0.910069,0.089931,0.910069
1,265,تنتظري أحدٍ يعطيكِ القوّة، قاتِلي واحصلي عليها...,1,1,0.999295,0.000705,0.999295,1,0.629949,0.370051,0.629949
2,3956,مهندسون معطلون ياهلا اذا تم القضاء بطالة المهن...,0,0,0.989365,0.989365,0.010635,0,0.969792,0.969792,0.030208
3,591,اسقاط الولاية؟؟ ، تسقط,0,1,0.954690,0.045309,0.954690,1,0.592428,0.407572,0.592428
4,2141,سعوديات نطلب اسقاط الولايه العالم مافي امرأة ع...,1,1,0.999885,0.000115,0.999885,1,0.999740,0.000260,0.999740


In [79]:
feature_cols = [
    'model1_predicted', 
    'model1_predicted_probability',
    'model1_prob_0',
    'model1_prob_1',
    'model2_predicted',
    'model2_predicted_probability',
    'model2_prob_0',
    'model2_prob_1'
]

X_meta_test = ensemble_testing_data[feature_cols].values
y_meta_test = ensemble_testing_data['Sentiment'].values

y_meta_test_pred = meta_learner.predict(X_meta_test)
y_meta_test_proba = meta_learner.predict_proba(X_meta_test)

test_acc = accuracy_score(y_meta_test, y_meta_test_pred)
test_f1 = f1_score(y_meta_test, y_meta_test_pred)
test_auc = roc_auc_score(y_meta_test, y_meta_test_proba[:, 1])

print(f"Test Accuracy:  {test_acc:.4f}")
print(f"Test F1-Score:  {test_f1:.4f}")
print(f"Test AUC:       {test_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_meta_test, y_meta_test_pred))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_meta_test, y_meta_test_pred)
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives:  {cm[1,1]}")


Test Accuracy:  0.9060
Test F1-Score:  0.9153
Test AUC:       0.9661

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       564
           1       0.92      0.91      0.92       712

    accuracy                           0.91      1276
   macro avg       0.90      0.91      0.90      1276
weighted avg       0.91      0.91      0.91      1276


Confusion Matrix:
[[508  56]
 [ 64 648]]

True Negatives:  508
False Positives: 56
False Negatives: 64
True Positives:  648


### 6a. Lets push this model to Vantage after conversion to ONNX

In [80]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Convert to ONNX
print("\nConverting to ONNX...")
# Specify the number of input features
n_features = X_meta_train.shape[1]  # Should be 8 (or however many features you used)

# Define the input type: [batch_size, n_features]
# Use None for batch_size to allow variable batch sizes
initial_type = [('input', FloatTensorType([None, n_features]))]

# Convert to ONNX
onnx_model = convert_sklearn(
    meta_learner,
    initial_types=initial_type,
    target_opset=16,
    dtype=np.float32
)

# Save ONNX model
output_path = "lr-meta.onnx"
with open(output_path, "wb") as f:
    f.write(onnx_model.SerializeToString())
    
print(f"ONNX model saved to: {output_path}")


Converting to ONNX...
ONNX model saved to: lr-meta.onnx


In [81]:
meta_file = "lr-meta.onnx"
meta_id = "lr-meta"
model_table_name = "arabic_sentiment_classifier"

In [82]:
tdml.save_byom(model_id=meta_id, model_file=meta_file, table_name=model_table_name)
tdml.list_byom(model_table_name)

Model is saved.
                                          model
model_id                                       
catb-model1-approach-4  b'8031208436174426F...'
lr-meta                 b'8081208736B6C326F...'
lr-model2-approach4     b'8081208736B6C326F...'


In [83]:
meta_onnx = tdml.retrieve_byom(model_id=meta_id, table_name=model_table_name)
meta_onnx

model_id,model
lr-meta,b'8081208736B6C326F...'


In [84]:
import onnx
m = onnx.load('lr-meta.onnx')
m.graph.input

[name: "input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
      }
      dim {
        dim_value: 8
      }
    }
  }
}
]

In [85]:
meta_results = tdml.ONNXPredict(
    newdata = full_test_data,
    modeldata = meta_onnx,
    accumulate = ["tid", "cleaned_doc", "Sentiment"],
    overwrite_cached_models = "*",
    model_input_fields_map = f"input=[3:]"
)

In [86]:
meta_results.result.to_sql("meta_learner_final_results", if_exists="replace", primary_index='tid')

In [87]:
tdml.execute_sql("SELECT TOP 1 * FROM meta_learner_final_results;").fetchall()

[[469,
  'الولاية عبودية ضد المرأة استغلها ذكور للتسلط عليها سعوديات نطلب اسقاط الولايه',
  1,
  '{"output_probability":[{"info":{"size":2,"keyType":"INT64","valueType":"FLOAT"},"value":{"0":0.03728497,"1":0.962715},"type":"ONNX_TYPE_MAP"}],"output_label":[1]}']]

In [88]:
try:
    tdml.db_drop_table("meta_test_set_final")
except:
    pass

query = """
CREATE TABLE meta_test_set_final AS (
SELECT
    tid,
    cleaned_doc,
    Sentiment,
    CAST(NEW JSON(json_report).JSONExtractValue('$.output_label[0]') AS INTEGER) AS meta_predicted,
    CASE 
        WHEN CAST(NEW JSON(json_report).JSONExtractValue('$.output_label[0]') AS INTEGER) = 0 
        THEN CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.0') AS FLOAT)
        ELSE CAST(NEW JSON(json_report).JSONExtractValue('$.output_probability[0].value.1') AS FLOAT)
    END AS meta_probability
FROM meta_learner_final_results
) WITH DATA
PRIMARY INDEX (tid);
"""

# Create a TeradataML DataFrame with the extracted probabilities
tdml.execute_sql(query)
result_with_probs_2 = tdml.DataFrame("meta_test_set_final")
result_with_probs_2.head()

tid,cleaned_doc,Sentiment,meta_predicted,meta_probability
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.8822799
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,1,0.8148439
16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابتلينا المسيار يهضم حقوق المرأه,1,0,0.62791705
19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا الحل النهائي حو حق حقوق المرأة وبيحل جزء المشكلة، الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة جرائم الشرف منتشرة دول ماعندها ولاية اكثر بكثير اللي عندنا نظرة تشاؤمية,1,1,0.9650933
27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حالياً مجلس الأمة الكويتي، مدافعة شرسة حقوق المرأة، وحق المواطن الكويتي وتمتلك عقلية اقتصادية مميزة أختيرت ضمن ٥٠ سيدة عربية مؤثرة مجتمعها,1,1,0.9638078
29,لأ كدا الفكرة اننا بلد مابتحترم حقوق الانسان اساساً حتحترم حقوق المرأة ؟ السودان ياجماعة ماتنسو,1,1,0.6984664
24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، الميكروفون، اللاسلكي، الراديو، السيكل، السيارة، التلفزيون، شوارع الإسفلت، الدش، الإنترنت، السينما، قيادة المرأة يقولون نقف ضد شيء لكنهم زالوا يقفون ضد المساواة الرجل المرأة نظرية التطور حقوق المثليين,1,0,0.86390513
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.88393784
6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.7637192
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.8000833


now lets chunk up the accuracy numbers

In [89]:
# First, let's get the confusion matrix components
meta_metrics_query = """
SELECT 
    COUNT(*) AS total,
    SUM(CASE WHEN meta_predicted = 1 AND Sentiment = 1 THEN 1 ELSE 0 END) AS TP,
    SUM(CASE WHEN meta_predicted = 0 AND Sentiment = 0 THEN 1 ELSE 0 END) AS TN,
    SUM(CASE WHEN meta_predicted = 1 AND Sentiment = 0 THEN 1 ELSE 0 END) AS FP,
    SUM(CASE WHEN meta_predicted = 0 AND Sentiment = 1 THEN 1 ELSE 0 END) AS FN,
    SUM(CASE WHEN meta_predicted = Sentiment THEN 1 ELSE 0 END) AS correct
FROM meta_test_set_final;
"""

# Get the confusion matrix values
confusion_val = tdml.DataFrame.from_query(meta_metrics_query).to_pandas()

total = confusion_val['total'].values[0]
TP = confusion_val['TP'].values[0]
TN = confusion_val['TN'].values[0]
FP = confusion_val['FP'].values[0]
FN = confusion_val['FN'].values[0]
correct = confusion_val['correct'].values[0]

# Calculate metrics
accuracy = correct / total
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


# Display results
print("="*60)
print("META MODEL PERFORMANCE")
print("="*60)
print(f"\nConfusion Matrix:")
print(f"  True Positives  (TP): {TP}")
print(f"  True Negatives  (TN): {TN}")
print(f"  False Positives (FP): {FP}")
print(f"  False Negatives (FN): {FN}")
print(f"\nMetrics:")
print(f"""  Accuracy:  {accuracy*100:.2f}%""")
print(f"""  Precision: {precision*100:.2f}%""")
print(f"""  Recall:    {recall*100:.2f}%""")
print(f"""  F1-Score:  {f1_score*100:.2f}%""")
print("="*60)


META MODEL PERFORMANCE

Confusion Matrix:
  True Positives  (TP): 648
  True Negatives  (TN): 508
  False Positives (FP): 56
  False Negatives (FN): 64

Metrics:
  Accuracy:  90.60%
  Precision: 92.05%
  Recall:    91.01%
  F1-Score:  91.53%


## 7. Cherry on Top

lets merge all the results in one table to get a better idea of how each model performed

In [90]:
# First, drop the table if it exists
try:
    tdml.db_drop_table("final_result_complete")
except:
    pass

# Create table with extracted probabilities from both models
create_query = """
CREATE TABLE final_result_complete AS (
    SELECT 
        m1.tid,
        m1.cleaned_doc,
        m1.Sentiment,
        m1.model1_predicted,
        m1.model1_predicted_probability,
        m1.model1_prob_0,
        m1.model1_prob_1,
        m1.model2_predicted,
        m1.model2_predicted_probability,
        m1.model2_prob_0,
        m1.model2_prob_1,
        m2.meta_predicted,
        m2.meta_probability
    FROM ensemble_test_data m1
    INNER JOIN meta_test_set_final m2
        ON m1.tid = m2.tid
) WITH DATA
PRIMARY INDEX (tid);
"""

# Execute the CREATE TABLE query
tdml.execute_sql(create_query)

# Load the table as a TeradataML DataFrame
final_results = tdml.DataFrame("final_result_complete")
final_results.head()

tid,cleaned_doc,Sentiment,model1_predicted,model1_predicted_probability,model1_prob_0,model1_prob_1,model2_predicted,model2_predicted_probability,model2_prob_0,model2_prob_1,meta_predicted,meta_probability
7,غسق سبيس حلوة الحلوات اشخاص هويتهم كيانهم المتفرد ، كونك تعرف الهوية فهذا يعطيك انت لاغيرك الحق بأن تهدم فكرة كبيرة كفكرة حقوق المرأة تنكر وجودها اثا,1,1,0.85138285,0.14861715,0.85138285,1,0.80144,0.19856003,0.80144,1,0.8822799
10,المرأة الإسلام منتقصة الحقوق والوااااااجبات وهذا منطقي وطبيعي لذلك أسوأ سيناريو الزواج مسلمة بلد أوروبي حقوق المواطنات واجباتهن,1,0,0.56588036,0.56588036,0.43411964,1,0.97415465,0.025845349,0.97415465,1,0.8148439
16,المسيار زي زواج المتعه الشيعه نعايب عليهم وابتلينا المسيار يهضم حقوق المرأه,1,0,0.8754652,0.8754652,0.124534786,1,0.8946601,0.105339885,0.8946601,0,0.62791705
19,عشان عندي تحفظ للي يشوف الغاء الولاية عندنا الحل النهائي حو حق حقوق المرأة وبيحل جزء المشكلة، الغيت الولاية سطوة المجتمع والعائلة ستظل موجودة جرائم الشرف منتشرة دول ماعندها ولاية اكثر بكثير اللي عندنا نظرة تشاؤمية,1,1,0.9325419,0.06745809,0.9325419,1,0.9947833,0.0052167475,0.9947833,1,0.9650933
27,صفاء الهاشم سيدة كويتية وهي المرأة الوحيدة حالياً مجلس الأمة الكويتي، مدافعة شرسة حقوق المرأة، وحق المواطن الكويتي وتمتلك عقلية اقتصادية مميزة أختيرت ضمن ٥٠ سيدة عربية مؤثرة مجتمعها,1,1,0.9882732,0.011726797,0.9882732,1,0.9284534,0.071546614,0.9284534,1,0.9638078
29,لأ كدا الفكرة اننا بلد مابتحترم حقوق الانسان اساساً حتحترم حقوق المرأة ؟ السودان ياجماعة ماتنسو,1,1,0.5005237,0.4994763,0.5005237,1,0.9183613,0.08163872,0.9183613,1,0.6984664
24,وقفوا ضد القهوة، الكرسي، الملعقة، الشوكة، الميكروفون، اللاسلكي، الراديو، السيكل، السيارة، التلفزيون، شوارع الإسفلت، الدش، الإنترنت، السينما، قيادة المرأة يقولون نقف ضد شيء لكنهم زالوا يقفون ضد المساواة الرجل المرأة نظرية التطور حقوق المثليين,1,0,0.79141575,0.79141575,0.20858425,0,0.6839293,0.6839293,0.3160707,0,0.86390513
8,مبدائيا انا حقوق المرأة وضد حقوق الرجل,1,1,0.78226686,0.21773314,0.78226686,1,0.87721,0.12278998,0.87721,1,0.88393784
6,اومن حقوق المرأة أنك تفتحلها الباب,1,1,0.99196845,0.008031547,0.99196845,0,0.7347209,0.7347209,0.2652791,1,0.7637192
2,لجنة التنمية بشبرا زال التسجيل مستمر دورة حقوق المرأة الطلاق وعيك يحميك,1,1,0.94781303,0.052186966,0.94781303,1,0.5667478,0.43325222,0.5667478,1,0.8000833


Lets compare each model for its performance

In [91]:
comparison_query = """
SELECT
    'Model 1' AS model,
    CAST(SUM(CASE WHEN model1_predicted = Sentiment THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS accuracy,
    SUM(CASE WHEN model1_predicted = Sentiment THEN 1 ELSE 0 END) AS correct,
    COUNT(*) AS total
FROM final_result_complete

UNION ALL

SELECT
    'Model 2',
    CAST(SUM(CASE WHEN model2_predicted = Sentiment THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*),
    SUM(CASE WHEN model2_predicted = Sentiment THEN 1 ELSE 0 END),
    COUNT(*)
FROM final_result_complete

UNION ALL

SELECT
    'Meta Learner',
    CAST(SUM(CASE WHEN meta_predicted = Sentiment THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*),
    SUM(CASE WHEN meta_predicted = Sentiment THEN 1 ELSE 0 END),
    COUNT(*)
FROM final_result_complete
"""

tdml.DataFrame.from_query(comparison_query).to_csv("model_comparison_results.csv", all_rows=True, fastexport=False)


Data is successfully exported into model_comparison_results.csv


# Clean-Up

In [92]:
tables_query = """
SELECT TableName 
FROM DBC.TablesV 
WHERE DatabaseName = 'demo_user' 
AND TableKind = 'T'
"""

tables_df = tdml.DataFrame.from_query(tables_query).to_pandas()
tables_df

,TableName
0,meta_learner_final_results
1,meta_test_set_final
2,ensemble_test_data
3,arabic_sentiment_classifier
4,model1_test_prediction_results
5,ensemble_train_data
6,model1_test_set_final
7,arabic_tweets_embedding_data
8,meta_learner_model1_test_results
9,sentiment_analysis_data_cleaned


In [93]:
# Drop each table
for table_name in tables_df['TableName']:
    try:
        tdml.db_drop_table(table_name)
        print(f"✓ Dropped table: {table_name}")
    except Exception as e:
        print(f"✗ Failed to drop {table_name}: {e}")

print("All tables dropped!")

✓ Dropped table: meta_learner_final_results
✓ Dropped table: meta_test_set_final
✓ Dropped table: ensemble_test_data
✓ Dropped table: arabic_sentiment_classifier
✓ Dropped table: model1_test_prediction_results
✓ Dropped table: ensemble_train_data
✓ Dropped table: model1_test_set_final
✓ Dropped table: arabic_tweets_embedding_data
✓ Dropped table: meta_learner_model1_test_results
✓ Dropped table: sentiment_analysis_data_cleaned
✓ Dropped table: model2_test_set_final
✓ Dropped table: final_result_complete
✓ Dropped table: model2_test_prediction_results
✓ Dropped table: meta_learner_model2_train_results
✓ Dropped table: meta_learner_model1_train_results
All tables dropped!


In [94]:
tdml.remove_context()

True